# Modeling and Evaluation

In this step, the preprocessed dataset is used to train several models. These are evaluated in terms of their accuracy based on several levels of feature selection.

Based on the [competetive ML survey](https://mlcontests.com/state-of-competitive-machine-learning-2022/#tabular-data), it is discussed that gradient-boosted tree based models are effective for tabular data. Therefore, the selection of hypothesis classes includes the `XGBoost` and `LightGBM` classes of models, but also the defacto `RandomForest` model, and the `SVM` and `MLP` models for some more variation in the inductive bias. The list of models is therefore:
- XGBoost
- LightGBM 
- RandomForest
- SVM
- MLP

https://arxiv.org/abs/2207.08815

https://arxiv.org/abs/2110.01889

https://arxiv.org/abs/2106.03253

https://github.com/dmlc/xgboost

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

pd.set_option('display.max_columns', None)

## 1. Baseline models

Training and evaluating baseline models without feature engineering

In [21]:
df = pd.read_csv('../datasets/data_cleaned.csv')

df = df.drop(columns=['R_fighter', 'B_fighter'])

df_dummies = pd.get_dummies(df.select_dtypes(exclude='number'))
df_num = df.select_dtypes(include='number')

df_y = df_num['Winner']
df_num = df_num.drop(columns=['Winner'])
df_num = (df_num - df_num.mean()) / df_num.std()

df_X = pd.concat([df_num, df_dummies], axis=1)
df_X

,R_age,R_Weight_lbs,R_Reach_cms,R_Height_cms,B_age,B_Weight_lbs,B_Reach_cms,B_Height_cms,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,title_bout,location_Argentina,location_Australia,location_Brazil,location_Canada,location_Chile,location_China,location_Croatia,location_Czech Republic,location_Denmark,location_France,location_Germany,location_Ireland,location_Japan,location_Mexico,location_Netherlands,location_New Zealand,location_Philippines,location_Poland,location_Russia,location_Singapore,location_South Korea,location_Sweden,location_USA,location_United Arab Emirates,location_United Kingdom,location_Uruguay,weight_class_Bantamweight,weight_class_CatchWeight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_LightHeavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Welterweight,weight_class_WomenBantamweight,weight_class_WomenFeatherweight,weight_class_WomenFlyweight,weight_class_WomenStrawweight,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
0,1.134909,1.162383,0.953000,1.665459,0.837676,1.198799,1.002779,0.835467,1.666785,-0.525529,0.445741,-0.041794,-1.022802,-0.961791,-0.613138,-0.588474,-0.441771,-0.420583,0.137251,-0.023796,0.118100,0.215608,-0.206084,-0.538141,-0.160302,-0.263355,-0.917332,-0.815576,-0.567493,-0.808243,0.178763,-0.084196,0.075739,0.047019,-0.125746,-0.028030,0.109261,0.314548,0.095739,0.169767,0.282378,0.395417,0.304518,0.233179,0.350020,0.622911,-0.419419,-0.354030,-0.627178,-0.597308,-0.545034,-0.573595,

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=.2, stratify=df_y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4911, 180), (1228, 180), (4911,), (1228,))

In [23]:
y_train

3078   -1
5509    1
2407    1
3257   -1
4840    1
       ..
3397    1
5698    1
4885    1
4911    1
3801    1
Name: Winner, Length: 4911, dtype: int64

In [25]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier().fit(X_train, y_train)

rf.score(X_test, y_test)

0.6392508143322475

In [26]:
from sklearn.svm import SVC

svm = SVC().fit(X_train, y_train)

svm.score(X_test, y_test)

0.6669381107491856

## 2. Applying Feature Engineering Techniques

- 